In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#USE FOR GOOGLE COLAB ONLY
train = pd.read_csv("/content/train.csv", sep = ',')
#train
test = pd.read_csv("/content/test.csv", sep = ',')
gender = pd.read_csv('/content/gender_submission.csv', sep = ',')
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [ ]:
test = pd.merge(gender, test, how = 'inner', on = 'PassengerId')
titanic = train.append(test)


In [ ]:
from sklearn.utils import shuffle
titanic = shuffle(titanic, random_state=789)

gender = pd.get_dummies(titanic['Sex'])
titanic = pd.concat((titanic, gender), axis=1)
titanic = titanic.drop(["Sex"], axis=1)
titanic = titanic.drop(["male"], axis=1)
titanic = titanic.rename(columns={"female": "Sex"})

titanic2 = titanic[['Pclass','Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]
titanic2 = titanic2.dropna(subset=['Age','Fare'])

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
titanic_scl = scaler.fit_transform(titanic2.to_numpy())

titanic_scl = pd.DataFrame(titanic_scl, columns = ['Pclass','Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived'])
titanic_scl

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,0.0,0.0,0.674308,0.125,0.000000,0.108215,0.0
1,0.0,0.0,0.448829,0.000,0.000000,0.051505,1.0
2,0.0,0.0,0.699361,0.000,0.000000,0.051822,0.0
3,1.0,0.0,0.002004,0.000,0.333333,0.028107,0.0
4,1.0,0.0,0.035450,0.500,0.333333,0.061264,1.0
...,...,...,...,...,...,...,...
1040,0.0,0.0,0.636728,0.000,0.166667,0.119804,0.0
1041,0.5,0.0,0.436302,0.000,0.000000,0.050749,0.0
1042,1.0,0.0,0.179506,1.000,0.333333,0.135753,0.0
1043,1.0,0.0,0.298509,0.000,0.000000,0.013761,0.0


In [ ]:
from sklearn.utils import shuffle
titanic_sck = shuffle(titanic_scl, random_state=123)
titanic_scl

from sklearn.model_selection import train_test_split
train, test, = train_test_split(titanic_scl, test_size = .20, random_state=379)

#train
#test

In [ ]:
#split data into 10 parts
folds = np.array_split(train, 10)

#

#list of accuracies from each fold
acc = []
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
# model using the default parameters
logreg = LogisticRegression()

for i in range(0, 10):
  print(i)
  #combine all the folds excpet the ith as training
  train_data = pd.concat(folds[:i] + folds[i+1:])
  #print("train data: " + str(train_data))
  #index predictor columns as x values
  train_x = train_data.iloc[:,0:6]
  print("train predictors: " + str(train_x))
  #last column = survived, use as y values
  train_y = train_data.iloc[:,6]
  print("train outcomes: " + str(train_y))
  #fit x and y values
  log_fit = logreg.fit(train_x, train_y)
  #test on ith fold (test fold)
  yhat = log_fit.predict(folds[i].iloc[:, 0:6])
  accuracy = metrics.accuracy_score(folds[i].iloc[:, 6], yhat)
  accuracy_percentage = 100 * accuracy
  print("fit accuracy of n-fold: " + str(accuracy_percentage))
  #take accuracy of prediction and append to list 
  acc.append(accuracy_percentage)

print("Mean accuracy of logistic regression by n-fold cross validation: %" + str(np.mean(acc)))

0
train predictors:       Pclass  Sex       Age  SibSp     Parch      Fare
315      1.0  0.0  0.448829  0.000  0.000000  0.015412
682      0.5  1.0  0.361142  0.125  0.000000  0.050749
973      1.0  1.0  0.373669  0.125  0.166667  0.047138
1028     0.0  0.0  0.711888  0.125  0.000000  0.285990
240      1.0  0.0  0.298509  0.000  0.000000  0.015713
...      ...  ...       ...    ...       ...       ...
35       0.0  1.0  0.523988  0.000  0.000000  0.444099
111      0.0  0.0  0.210823  0.000  0.000000  0.091933
163      0.5  1.0  0.223350  0.000  0.166667  0.044893
231      0.0  0.0  0.448829  0.000  0.166667  1.000000
752      1.0  0.0  0.636728  0.000  0.000000  0.013769

[752 rows x 6 columns]
train outcomes: 315     0.0
682     1.0
973     0.0
1028    0.0
240     0.0
       ... 
35      1.0
111     0.0
163     1.0
231     1.0
752     0.0
Name: Survived, Length: 752, dtype: float64
fit accuracy of n-fold: 83.33333333333334
1
train predictors:      Pclass  Sex       Age  SibSp     Parc

mean accuracy of logistic regression by k-fold cross validation: %84.45352839931154


In [ ]:
#grid search

folds = np.array_split(train, 10)
penalty = ['l1', 'l2']
C = [0.001, 0.01, 0.1, 1, 10, 100]


acc_grid = []
#for every penalty
for p in range(len(penalty)):
  #for every learning rate
  for c in range(len(C)):
    #create new logistic regression model with each unique combination of hyperparameters
    logreg_grid = LogisticRegression(penalty = penalty[p], C = C[c], solver = 'saga')
    acc_grid.append("ACC with penalty:" + penalty[p] + " and C: " + str(C[c]))
    perf = []
    for i in range(0, 10):
      trainfolds = pd.concat(folds[:i] + folds[i+1:])
      train_x = trainfolds.iloc[:,0:6]
      train_y = trainfolds.iloc[:,6]
      model_grid = logreg_grid.fit(train_x, train_y)
      yhat = model_grid.predict(folds[i].iloc[:, 0:6])
      accuracy = metrics.accuracy_score(folds[i].iloc[:, 6], yhat)
      accuracy_percentage = 100 * accuracy
        #print(accuracy_percentage)
      perf.append(accuracy_percentage)
    best = max(perf)
    acc_grid.append(best)
        
acc_grid

['ACC with penalty:l1 and C: 0.001',
 65.47619047619048,
 'ACC with penalty:l1 and C: 0.01',
 65.47619047619048,
 'ACC with penalty:l1 and C: 0.1',
 89.1566265060241,
 'ACC with penalty:l1 and C: 1',
 89.1566265060241,
 'ACC with penalty:l1 and C: 10',
 88.09523809523809,
 'ACC with penalty:l1 and C: 100',
 88.09523809523809,
 'ACC with penalty:l2 and C: 0.001',
 65.47619047619048,
 'ACC with penalty:l2 and C: 0.01',
 85.71428571428571,
 'ACC with penalty:l2 and C: 0.1',
 89.1566265060241,
 'ACC with penalty:l2 and C: 1',
 89.1566265060241,
 'ACC with penalty:l2 and C: 10',
 87.95180722891565,
 'ACC with penalty:l2 and C: 100',
 88.09523809523809]

In [ ]:
#Compare with testing 
logreg_grid_test = LogisticRegression(penalty = 'l1', C = 0.1, solver = 'saga')
model_grid_test = logreg_grid_test.fit(train.iloc[:, 0:6], train.iloc[:,6])
yhat_test = model_grid_test.predict(test.iloc[:, 0:6])
accuracy_test = metrics.accuracy_score(test.iloc[:, 6], yhat_test)
accuracy_percentage_test = 100 * accuracy_test
print("accuracy of logistic w/ best hyperparamters on test: " + str(accuracy_percentage_test))


accuracy of logistic w/ best hyperparamters on test: 87.08133971291866
